In [1]:
%pip install git+https://github.com/royerlab/tracksdata

  Cloning https://github.com/royerlab/tracksdata to /private/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/pip-req-build-bq1udb3u
  Running command git clone --filter=blob:none --quiet https://github.com/royerlab/tracksdata /private/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/pip-req-build-bq1udb3u
  Resolved https://github.com/royerlab/tracksdata to commit 2c5a21ad5af29bdf1574e678c4f615b7fbd3e8d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/qiskit/rustworkx.git (to revision ac1082ca9d1774e6e9a91436a15a201b0c7baffd) to /private/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/pip-install-t5g8trb_/rustworkx_3722aa30fe1b435bace861c89238c85c
  Running command git clone --filter=blob:none --quiet https://github.com/qiskit/rustworkx.git /private/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/pip-install-t5g8trb_/rustworkx_3722aa30fe1b435bace861c89238c85c
  Running com

In [2]:
import tracksdata as td